In [ ]:
# Load in our libraries
import pandas as pd
import numpy as np
import sklearn
import xgboost as xgb
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import KFold
plt.style.use('seaborn')
sns.set(font_scale=2)


%matplotlib inline
import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.nn.init
from torch.utils.data import Dataset, DataLoader,TensorDataset,random_split,SubsetRandomSampler, ConcatDataset
import torch
from torchvision import transforms
from PIL import Image

<ipython-input-1-a7f0f56daf37>:9: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [ ]:
#CUDA 메모리 할당 / 디바이스 할당
USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
df = pd.read_pickle('/content/gdrive/My Drive/LSWMD.pkl')

Mounted at /content/gdrive


Data 전처리

In [ ]:
# 웨이퍼 맵 => 많은 정보는 없지만 인스턴스 별로 다이 크기가 다름.
# 웨이퍼 맵 차원 검사를 위한 새로운 변수 WaferMapDim 생성
def find_dim(x):
    dim0=np.size(x,axis=0)
    dim1=np.size(x,axis=1)
    return dim0,dim1
df['waferMapDim']=df.waferMap.apply(find_dim)
df.sample(5)

,waferMap,dieSize,lotName,waferIndex,trianTestLabel,failureType,waferMapDim
342361,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,...",2570.0,lot20521,25.0,[],[],"(60, 55)"
217010,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2,...",599.0,lot13765,7.0,[],[],"(28, 29)"
514095,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1,...",515.0,lot32116,16.0,[],[],"(25, 27)"
399238,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,...",1091.0,lot23885,5.0,[],[],"(37, 37)"
610990,"[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,...",710.0,lot38410,23.0,[],[],"(32, 29)"


In [ ]:
# 고장 유형 확인 and class 분류
df['failureNum'] = df.failureType
mapping_type={'Center':0,'Donut':1,'Edge-Loc':2,'Edge-Ring':3,'Loc':4,'Random':5,'Scratch':6,'Near-full':7,'none':8}
df=df.replace({'failureNum':mapping_type})

/usr/local/lib/python3.10/dist-packages/pandas/core/array_algos/replace.py:86: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  op = lambda x: operator.eq(x, b)


In [ ]:
#df_withlabel (라벨포함) : 라벨링된 웨이퍼
#df_withpattern (패턴포함): 라벨링&패턴 웨이퍼
#df_nonpattern (패턴미포함) : 라벨링은 됐지만 패턴은 없는 웨이퍼

df_withlabel = df[(df['failureType']!=0)] # 라벨링된 웨이퍼 : failureType = 0이 아닌 df
df_withlabel =df_withlabel.reset_index() #labeled index.

df_withpattern = df_withlabel[(df_withlabel['failureType'] != 'none')]  # 라벨링&패턴 웨이퍼: failureType = none 이 아닌 df
df_withpattern = df_withpattern.reset_index() #patterned index.

df_nonpattern = df_withlabel[(df_withlabel['failureType'] == 'none')] # 라벨은 있고 패턴은 없는 웨이퍼:  failureType == none 인 df
df_withlabel.shape[0], df_withpattern.shape[0], df_nonpattern.shape[0]


/usr/local/lib/python3.10/dist-packages/pandas/core/ops/array_ops.py:75: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = libops.scalar_compare(x.ravel(), y, op)


(172950, 25519, 147431)

In [ ]:
#셔플
df_withpattern = df_withpattern.sample(frac=1).reset_index(drop=True)

In [ ]:
tensor_data=df_withpattern['waferMap']
print(tensor_data.shape)

(20413,)
(5106,)


Resize / RGb

In [ ]:
#여러 개의 웨이퍼 크기를 통일하기 위한 resize 함수
def resize(data, Dim0, Dim1):
    transTensor = transforms.ToTensor()
    resizer = transforms.Resize((Dim0, Dim1))
    rwm = torch.ones((1, Dim0, Dim1)).to(DEVICE)
    for i in range(len(data)):
        a = transTensor(data[i].astype('int32'))
        a = resizer(a).reshape((1, Dim0, Dim1)).to(DEVICE)
        rwm = torch.cat([rwm, a])
    x = rwm[1:]
    del rwm
    return x

In [ ]:
#텐서 데이터의 크기를 resize함수를 통해 64,64로 통일
resized_images = resize(tensor_data, 64, 64)

/usr/local/lib/python3.10/dist-packages/torchvision/transforms/functional.py:1603: UserWarning: The default value of the antialias parameter of all the resizing transforms (Resize(), RandomResizedCrop(), etc.) will change from None to True in v0.17, in order to be consistent across the PIL and Tensor backends. To suppress this warning, directly pass antialias=True (recommended, future default), antialias=None (current default, which means False for Tensors and True for PIL), or antialias=False (only works on Tensors - PIL will still use antialiasing). This also applies if you are using the inference transforms from the models weights: update the call to weights.transforms(antialias=True).
  warnings.warn(


In [ ]:
# 3차원의 데이터로 만들기 위한 RGB함수
# 0: 웨이퍼가 아닌 부분 -> R, 1: 정상 웨이퍼 부분 -> G, 2: 결함있는 부분 -> B|

def rgb_sw(x): #rgb 추가
  Dim0 = np.size(x, axis=1)
  Dim1 = np.size(x, axis=2)
  new_x = np.zeros((len(x), Dim0, Dim1, 3)) # 0으로만 이루어진 array 생성
  x = torch.unsqueeze(x,-1) # 차원이 1인 차원 생성
  x = x.to(torch.device('cpu'))
  x = x.numpy()
  for w in range(len(x)):
      for i in range(Dim0):
          for j in range(Dim1):
              new_x[w, i, j, int(x[w, i, j])] = 1
  return new_x

resized_images = rgb_sw(resized_images)

In [ ]:
#추가 된 채널 확인
print(resized_images.shape)

(20413, 64, 64, 3)
(5106, 64, 64, 3)


In [ ]:
#데이터 사용을 위해 텐서데이터로 변경
resized_wm = torch.Tensor(np.transpose(resized_images, (0, 3, 1, 2)))
print(resized_wm.shape)

torch.Size([20413, 3, 64, 64])
torch.Size([5106, 3, 64, 64])


In [ ]:
#라벨 및 클래스 지정 및 확인
#레이블 변수 지정
labels = df_withpattern['failureType']
labels = labels.apply(lambda x: x[0][0])
labels = labels.replace({'': 'none'})
labels = labels.to_numpy()
labels = np.array(labels).reshape((-1,1))
label_wm = labels

faulty_case = np.unique(labels).flatten().tolist()
np.array(labels, dtype='<U8')
print(faulty_case)

['Center', 'Donut', 'Edge-Loc', 'Edge-Ring', 'Loc', 'Near-full', 'Random', 'Scratch']
